In [1]:
!pip install transformers fastapi uvicorn spacy scikit-learn pandas torch pydantic
!python -m spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [1]:
from google.colab import files
uploaded = files.upload()


Saving combined_emails_with_natural_pii.csv to combined_emails_with_natural_pii.csv


In [2]:
import pandas as pd

# Replace this with the exact filename printed after upload
df = pd.read_csv("combined_emails_with_natural_pii.csv")

df.head()


,email,type
0,Subject: Unvorhergesehener Absturz der Datenan...,Incident
1,Subject: Customer Support Inquiry\n\nSeeking i...,Request
2,Subject: Data Analytics for Investment\n\nI am...,Request
3,Subject: Krankenhaus-Dienstleistung-Problem\n\...,Incident
4,"Subject: Security\n\nDear Customer Support, I ...",Request


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('email').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('type').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['type'].value_counts()
    for x_label, grp in _df_2.groupby('email')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('email')
_ = plt.ylabel('type')

In [3]:
print(df.columns)


Index(['email', 'type'], dtype='object')


In [4]:
import re
import spacy

nlp = spacy.load("en_core_web_sm")

PII_PATTERNS = {
    "email": r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+',
    "phone_number": r'\b\d{10}\b',
    "aadhar_num": r'\b\d{4} \d{4} \d{4}\b',
    "credit_debit_no": r'\b(?:\d[ -]*?){13,16}\b',
    "cvv_no": r'\b\d{3}\b',
    "expiry_no": r'\b(0[1-9]|1[0-2])\/?([0-9]{2})\b',
    "dob": r'\b\d{2}[-/]\d{2}[-/]\d{4}\b',
}

def mask_pii(text):
    masked_text = text
    entities = []

    # Regex masking
    for label, pattern in PII_PATTERNS.items():
        for match in re.finditer(pattern, text):
            span = match.span()
            original = text[span[0]:span[1]]
            masked_text = masked_text.replace(original, f"[{label}]")
            entities.append({"position": list(span), "classification": label, "entity": original})

    # Name entity masking with spaCy
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            masked_text = masked_text.replace(ent.text, "[full_name]")
            entities.append({"position": [ent.start_char, ent.end_char], "classification": "full_name", "entity": ent.text})

    return masked_text, entities


In [6]:
print(df.columns.tolist())


['email', 'type']


In [8]:
import pandas as pd

df = pd.read_csv("combined_emails_with_natural_pii.csv")
print("📋 Columns in your dataset:")
print(df.columns.tolist())
print("\n🔍 First few rows:")
print(df.head())


📋 Columns in your dataset:
['email', 'type']

🔍 First few rows:
                                               email      type
0  Subject: Unvorhergesehener Absturz der Datenan...  Incident
1  Subject: Customer Support Inquiry\n\nSeeking i...   Request
2  Subject: Data Analytics for Investment\n\nI am...   Request
3  Subject: Krankenhaus-Dienstleistung-Problem\n\...  Incident
4  Subject: Security\n\nDear Customer Support, I ...   Request


In [9]:
['EmailContent', 'Type']


['EmailContent', 'Type']

In [11]:
import pandas as pd

df = pd.read_csv("combined_emails_with_natural_pii.csv")

print("🧾 Actual column names:")
print(df.columns.tolist())

print("\n🧪 Sample data:")
print(df.head(3).to_string())


🧾 Actual column names:
['email', 'type']

🧪 Sample data:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  email      type
0                                                                                                                            

In [14]:
def classify_email(text):
    masked_email, entities = mask_pii(text)
    inputs = tokenizer(masked_email, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=1)
        label = label_enc.inverse_transform(prediction.cpu().numpy())[0]

    return {
        "input_email_body": text,
        "list_of_masked_entities": entities,
        "masked_email": masked_email,
        "category_of_the_email": label
    }



In [16]:
!pip install fastapi uvicorn pydantic


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.7 MB/s eta 0:00:00


In [17]:
!pip install nest-asyncio
import nest_asyncio
import uvicorn

nest_asyncio.apply()


In [19]:
print(df.columns.tolist())


['email', 'type']


In [20]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class EmailIn(BaseModel):
    input_email_body: str

@app.post("/classify")
def classify_endpoint(data: EmailIn):
    return classify_email(data.input_email_body)
